In [ ]:
import time

from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange import get_exchange, Market
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun import cl

import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config


# 市场
market = 'a'  # a 沪深A股  hk 港股 us 美股 futures 期货 currency 数字货币
# 标的代码
code = 'SHSE.000300'
# 获取周期，可设置多个
frequencys = ['1m']
# frequencys = ['30m','5m']
# 开始时间（需要api接口支持）
start_date = "2025-02-25 09:30:00"
# 结束时间（需要api接口支持）
end_date = None
# 缠论计算配置
cl_config = query_cl_chart_config(market, code)
print(cl_config)
# ex = get_exchange(Market(market)) # 使用在线的k线数据接口
ex = ExchangeDB(market) # 读取数据库中的k线数据

In [ ]:
df = klines[['date','open','high','low','close','volume']]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import time

from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange import get_exchange, Market
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun import cl

import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config


# 市场
market = 'a'  # a 沪深A股  hk 港股 us 美股 futures 期货 currency 数字货币
# 标的代码
code = 'SHSE.000300'
# 获取周期，可设置多个
frequencys = ['1m']
# frequencys = ['30m','5m']
# 开始时间（需要api接口支持）
start_date = "2025-02-25 09:30:00"
# 结束时间（需要api接口支持）
end_date = None
# 缠论计算配置
cl_config = query_cl_chart_config(market, code)
print(cl_config)
# ex = get_exchange(Market(market)) # 使用在线的k线数据接口
ex = ExchangeDB(market) # 读取数据库中的k线数据

klines = ex.klines(code, frequency=frequencys, start_date=start_date, end_date=end_date)
# 假设 klines 是一个已经定义的 DataFrame
df = klines[['date','open','high','low','close','volume']]

# 确保 'date' 列是 datetime 类型
df['date'] = pd.to_datetime(df['date'])

# 设置 'date' 列为索引
df.set_index('date', inplace=True)

# 绘制 K 线图
mpf.plot(df, type='candle', volume=False, show_nontrading=False)

# 加权计算average_price
weights = {'open': 0.2, 'close': 0.2, 'high': 0.1, 'low': 0.1, 'open_close_avg': 0.2, 'all_avg': 0.2}
df['open_close_avg'] = (df['open'] + df['close']) / 2
df['all_avg'] = (df['open'] + df['close'] + df['high'] + df['low']) / 4

df['average_price'] = (df['open'] * weights['open'] +
                       df['close'] * weights['close'] +
                       df['high'] * weights['high'] +
                       df['low'] * weights['low'] +
                       df['open_close_avg'] * weights['open_close_avg'] +
                       df['all_avg'] * weights['all_avg'])

# 计算价格区间和成交量分布
bin_cnt = 100  # 默认间隔为1
min_price = df['average_price'].min()
max_price = df['average_price'].max()
interval = (max_price - min_price) / bin_cnt  # 确保 interval 是浮点数
bins = [min_price + i * interval for i in range(bin_cnt + 1)]
labels = [f'{bins[i]:.2f}-{bins[i+1]:.2f}' for i in range(len(bins) - 1)]

df['price_range'] = pd.cut(df['average_price'], bins=bins, labels=labels, right=False)
volume_by_price_range = df.groupby('price_range')['volume'].sum().reset_index()

# 绘制横向 FRVP 图表
plt.barh(volume_by_price_range['price_range'], volume_by_price_range['volume'], color='blue', alpha=0.7)

# 调整布局
plt.tight_layout()
plt.show()

In [ ]:
renders = []
cl_datas = []


for f in frequencys:
    klines = ex.klines(code, frequency=f, start_date=start_date, end_date=end_date)
    klines = klines[:]
    print(f, '获取K线数据量：', len(klines))
    _s = time.time()
    cl_datas.append(cl.CL(code, f, cl_config).process_klines(klines[:]))
    print('Run time: ', time.time() - _s)

for cd in cl_datas:
    title = '%s - 【%s】 周期数据图表' % (code, cd.frequency)
    renders.append(kcharts.render_charts(title, cd))

In [ ]:
klines

In [ ]:
print(klines[:-798].tail(1))
klines[:-797].tail(1)

In [ ]:
renders[0]

In [ ]:
renders = []
cl_datas = []


for f in frequencys:
    klines = ex.klines(code, frequency=f, start_date=start_date, end_date=end_date)
    klines = klines[:]
    print(f, '获取K线数据量：', len(klines))
    _s = time.time()
    cl_datas.append(cl.CL(code, f, cl_config).process_klines(klines[:-797]))
    print('Run time: ', time.time() - _s)

for cd in cl_datas:
    title = '%s - 【%s】 周期数据图表' % (code, cd.frequency)
    renders.append(kcharts.render_charts(title, cd))